In [8]:
import numpy as np
import pandas as pd

In [ ]:
# data = pd.read_csv('./Data/CSD Generation (Hourly) - 2020-01 to 2020-06/CSD Generation (Hourly) - 2020-01 to 2020-06.csv')
# renewable_data = data.loc[(data['Fuel Type'] == 'WIND') | (data['Fuel Type'] == 'SOLAR')]
# area_data = renewable_data.loc[renewable_data['Planning Area'] == 52]
# area_data["Fuel Type"].unique()
# area_data["Asset Name"].unique()

## Weather Data

In [17]:
vauxhaul1 = pd.read_csv('./Data/ACIS/Vauxhall-20200101-20200601.csv', encoding='unicode_escape')
vauxhaul2 =  pd.read_csv('./Data/ACIS/Vauxhall-20200701-20201231.csv', encoding='unicode_escape')
vauxhaul3 =  pd.read_csv('./Data/ACIS/Vauxhall-20210101-20210630.csv', encoding='unicode_escape')
vauxhaul4 =  pd.read_csv('./Data/ACIS/Vauxhall-20210701-20211231.csv', encoding='unicode_escape')

vauxhaul = pd.concat([vauxhaul1, vauxhaul2, vauxhaul3, vauxhaul4])

## Generation Data

In [20]:
Gen1 = pd.read_csv('./Data/CSD/CSD Generation (Hourly) - 2020-01 to 2020-06.csv')
Gen2 = pd.read_csv('./Data/CSD/CSD Generation (Hourly) - 2020-07 to 2020-12.csv')
Gen3 = pd.read_csv('./Data/CSD/CSD Generation (Hourly) - 2021-01 to 2021-06.csv')
Gen4 = pd.read_csv('./Data/CSD/CSD Generation (Hourly) - 2021-07 to 2021-12.csv')

GenData = pd.concat([Gen1, Gen2, Gen3, Gen4])

In [23]:
vauxhaul_Gen = GenData.loc[GenData['Asset Name'] == 'VXH1 Vauxhall']

In [24]:
vauxhaul_Gen.head()

,Date (MST),Date (MPT),Asset Short Name,Asset Name,Asset Grouping,Volume,Maximum Capability,System Capability,Fuel Type,Sub Fuel Type,Planning Area,Region
177120,2020-02-19 01:00:00,2020-02-19 01:00:00,VXH1,VXH1 Vauxhall,VXH1,0.0,22.0,22.0,SOLAR,SOLAR,52,South
177121,2020-02-19 02:00:00,2020-02-19 02:00:00,VXH1,VXH1 Vauxhall,VXH1,0.0,22.0,22.0,SOLAR,SOLAR,52,South
177122,2020-02-19 03:00:00,2020-02-19 03:00:00,VXH1,VXH1 Vauxhall,VXH1,0.0,22.0,22.0,SOLAR,SOLAR,52,South
177123,2020-02-19 04:00:00,2020-02-19 04:00:00,VXH1,VXH1 Vauxhall,VXH1,0.0,22.0,22.0,SOLAR,SOLAR,52,South
177124,2020-02-19 05:00:00,2020-02-19 05:00:00,VXH1,VXH1 Vauxhall,VXH1,0.0,22.0,22.0,SOLAR,SOLAR,52,South


In [25]:
vauxhaul.head()

,Station Name,Date (Local Standard Time),Air Temp. Inst. (°C),Air Temp. Inst. Source Flag,Air Temp. Inst. Comment,Humidity Inst. (%),Humidity Inst. Source Flag,Humidity Inst. Comment,Relative Humidity Avg. (%),Relative Humidity Avg. Source Flag,...,Wind Speed 10 m Syno. Comment,Wind Dir. 10 m Syno. (°),Wind Dir. 10 m Syno. Source Flag,Wind Dir. 10 m Syno. Comment,Wind Speed 10 m Avg. (km/h),Wind Speed 10 m Avg. Source Flag,Wind Speed 10 m Avg. Comment,Wind Dir. 10 m Avg. (°),Wind Dir. 10 m Avg. Source Flag,Wind Dir. 10 m Avg. Comment
0,Vauxhall CDA CS,01-January-2020 00:00,2.5,ACTUAL,NaN,74.0,ACTUAL,NaN,76.0,ACTUAL,...,NaN,271.0,ACTUAL,NaN,22.2,ACTUAL,NaN,271.0,ACTUAL,NaN
1,Vauxhall CDA CS,01-January-2020 01:00,2.5,ACTUAL,NaN,75.0,ACTUAL,NaN,75.0,ACTUAL,...,NaN,272.0,ACTUAL,NaN,22.8,ACTUAL,NaN,272.0,ACTUAL,NaN
2,Vauxhall CDA CS,01-January-2020 02:00,2.4,ACTUAL,NaN,77.0,ACTUAL,NaN,77.0,ACTUAL,...,NaN,273.0,ACTUAL,NaN,21.6,ACTUAL,NaN,269.0,ACTUAL,NaN
3,Vauxhall CDA CS,01-January-2020 03:00,2.8,ACTUAL,NaN,78.0,ACTUAL,NaN,78.0,ACTUAL,...,NaN,277.0,ACTUAL,NaN,18.7,ACTUAL,NaN,275.0,ACTUAL,NaN
4,Vauxhall CDA CS,01-January-2020 04:00,0.9,ACTUAL,NaN,83.0,ACTUAL,NaN,81.0,ACTUAL,...,NaN,349.0,ACTUAL,NaN,13.5,ACTUAL,NaN,335.0,ACTUAL,NaN


In [32]:
vauxhaul_Gen["timestamp"] = pd.to_datetime(vauxhaul_Gen["Date (MPT)"]).dt.date
vauxhaul["timestamp"] = pd.to_datetime(vauxhaul["Date (Local Standard Time)"]).dt.date

vauxhaul_Gen.to_csv('./Data/CSD/VauxhallGen.csv')
vauxhaul.to_csv('./Data/ACIS/Vauxhall.csv')

C:\Users\nabil\AppData\Local\Temp\ipykernel_27008\1913567382.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vauxhaul_Gen["timestamp"] = pd.to_datetime(vauxhaul_Gen["Date (MPT)"]).dt.date
